# Dynamic Differential Microscopy

See the readme file for more information.

## Analysis
Edit the variables in the first cell and run. This will store output in the file FTOneDSlices.txt.

* Binsize - Bin size in pixels for the histogram used in the radial averaging of the Fourier transform (FT).
* cutoff - Maximum frame averaging for each timestep. Set to 0 for analysis of all possible time increments between all images.
* images_to_load - Number of timesteps to load from disk. Set to 0 for all available images.
* analysisradius - Radius of FT radial averaging. Set to 0 for analysis of full FT. Deceasing this value can greatly speed up processing since the number of pixels to average increases as radius^2.
* image_directory - Directory path of images to analyse, end with slash.
* file_prefix - Prefix for file. Program assumes files are numbered sequentially from 0 with 4 digits i.e. image_0000.png, image_0001.png...
* file_suffix - File type. Tested with tif and png.

In [ ]:
import ddm

binsize = 1
cutoff = 250
images_to_load = 0
analysisradius = 100
image_directory = "example_images/"
file_prefix = "iii_"
file_suffix = ".png"


ddm.main(binsize, cutoff, images_to_load, analysisradius, image_directory, file_prefix, file_suffix)

## Plotting
The analysis code will now load the file saved from the analysis and plot graphs. There are four plotting functions. These can be turned on and off by commenting or uncommenting them. If you have a local LaTeX distribution this will be used to plot graph axis labels. If not set use_latex to False. This will give functional but ugly labels.

For the example data series we can see a peak in the intensity of the Fourier difference around 30 inverse pixels. Plotting the Fourier Difference as a function of time for wavevectors between 25 and 30 pixels shows a platau at around 5 seconds. This corresponds well with the relaxation time determined from particle tracking.

In [ ]:
from ddm_data_processing import *
#from matplotlib import rc
%matplotlib inline


def plotmain():
    list_of_files = ["FTOneDSlices_octave1_part_0000.txt"]
    pixel_size = [40]
    particle_size = [200]
    frame_time = [1]
    wavevectors = (25, 30)
    use_latex = True
    
    if not use_latex:
        rc('text', usetex=False)
    
    data = Series(list_of_files, pixel_size, particle_size, frame_time)

    plot_azimuthal_angle(data, frame_number = 20, q_in_pixels=True)
    plot_wavevector_range(data.datasets[0], wavevectors)
    #plot_wavevector_samples(data, wavevector = 25)
    #plot_image(data.datasets[0])
    # Save the data cropped to the specified wavevetors, useful for plotting in Excel or fitting with another program.
    save_series(data, wavevectors)
    
plotmain()

# Data Fitting

Once the data is produced, the stretched exponentials of the relaxation can be fitted to give relaxation time as a function of q

In [ ]:
import ddm_fitting

timestep = 1                      # Spacing in seconds between frames
n_data_points = 100               # Fit only the first N timesteps

ddm_fitting.fit_curves(float(timestep), n_data_points, octave_start=1, octave_end=6)